In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [2]:
data = pd.read_csv("BTC-USD_SIGNALS.csv")

In [3]:
data

,Date,Open,High,Low,Close,Volume,Variation,RSI,MACD,MACD_H,Confirmation Time,Transactions,Miners Revenue,FnG
0,2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,-0.292146,-2.712213,-1.043523,-1.108359,-0.475907,1.144750,-0.774692,-0.186892,-0.583209
1,2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,-0.152676,-1.011481,-1.204172,-1.202571,-0.654534,1.144750,-0.774692,-0.186892,-1.244407
2,2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,-0.427993,0.955604,-1.632900,-1.235294,-0.618233,1.114249,-0.774692,-0.186892,-0.142410
3,2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,-0.437581,-2.584947,-1.626830,-1.320379,-0.741794,1.114249,-1.144752,-0.264496,-0.847688
4,2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,-0.326118,-4.184569,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2022-09-04,19832.470703,19999.689453,19636.816406,19986.712891,0.478227,0.148948,-1.113287,-0.844382,-0.254845,-1.063067,-0.648298,-0.307825,-1.024008
1680,2022-09-05,19988.789063,20031.160156,19673.046875,19812.371094,0.658018,-0.278482,-1.133325,-0.831832,-0.166540,-0.742360,-0.648298,-0.307825,-0.891768
1681,2022-09-06,19817.724609,20155.269531,18800.171875,18837.667969,1.393318,-1.325705,-1.548973,-0.892284,-0.308652,-0.742360,-0.165203,-0.359746,-0.935848
1682,2022-09-07,18837.683594,19427.171875,18644.466797,19290.324219,0.981876,0.568952,-1.180050,-0.893278,-0.249058,-0.742360,-0.165203,-0.359746,-0.847688


In [4]:
predict_data = data.copy().drop(["Open","Close","High","Low"],axis=1)
max_days = 5
target_range = 2
for i in range(1,max_days):#2jours
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [5]:
predict_data = predict_data[[i % int(max_days / 2) == 0 for i in range(len(predict_data))]]

In [6]:
predict_data

,Date,Volume,Variation,RSI,MACD,MACD_H,Confirmation Time,Transactions,Miners Revenue,FnG,...,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,CONF-4,TRANS-4,REV-4,FnG-4,Target
0,2018-02-05,-0.326118,-4.184569,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,...,-2.712213,-0.292146,-1.043523,-1.108359,-0.475907,1.144750,-0.774692,-0.186892,-0.583209,1
2,2018-02-07,-0.331965,-0.495589,-1.457407,-1.552132,-0.860381,0.453384,-1.959889,-0.596734,-0.318730,...,0.955604,-0.427993,-1.632900,-1.235294,-0.618233,1.114249,-0.774692,-0.186892,-0.142410,1
4,2018-02-09,-0.452131,1.422827,-0.830140,-1.412981,-0.171285,0.207605,-1.959889,-0.596734,0.033910,...,-4.184569,-0.326118,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,0
6,2018-02-11,-0.485525,-1.529055,-1.178450,-1.298888,0.178688,0.207605,-2.453448,-0.455877,-0.539129,...,-0.495589,-0.331965,-1.457407,-1.552132,-0.860381,0.453384,-1.959889,-0.596734,-0.318730,1
8,2018-02-13,-0.506967,-1.004265,-0.584168,-1.121564,0.570097,0.276565,-1.522212,-0.160496,-0.362809,...,1.422827,-0.452131,-0.830140,-1.412981,-0.171285,0.207605,-1.959889,-0.596734,0.033910,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,2022-08-28,0.433926,-0.601366,-2.100264,-0.799658,-1.290428,-1.263537,-1.414496,-0.174982,-0.671369,...,-0.212717,0.816704,-1.545836,-0.444016,-1.152727,-1.025714,-0.498097,-0.206563,-0.803608,1
1670,2022-08-30,0.943757,-0.691652,-1.653641,-0.871357,-1.020465,-1.313931,-1.414496,-0.174982,-0.715449,...,-1.658930,1.339033,-2.000380,-0.584031,-1.132733,-1.025714,-0.310257,-0.097684,-0.715449,1
1672,2022-09-01,0.726988,0.047069,-1.336325,-0.871305,-0.646449,-1.313931,-0.310919,-0.277036,-1.024008,...,-0.601366,0.433926,-2.100264,-0.799658,-1.290428,-1.263537,-1.414496,-0.174982,-0.671369,0
1674,2022-09-03,0.395940,-0.231177,-0.964656,-0.866931,-0.401653,-1.063067,-0.648298,-0.307825,-0.979928,...,-0.691652,0.943757,-1.653641,-0.871357,-1.020465,-1.313931,-1.414496,-0.174982,-0.715449,0


In [7]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date","Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date","Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date","Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [38]:
algo = "CAT"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(200,2000,200),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Learning rate set to 0.066538
0:	learn: 0.6638149	total: 1.29s	remaining: 2m 7s
1:	learn: 0.6414676	total: 2.23s	remaining: 1m 49s
2:	learn: 0.6176488	total: 3.13s	remaining: 1m 41s
3:	learn: 0.5951911	total: 3.98s	remaining: 1m 35s
4:	learn: 0.5793617	total: 5.06s	remaining: 1m 36s
5:	learn: 0.5622152	total: 6.24s	remaining: 1m 37s
6:	learn: 0.5485485	total: 7.42s	remaining: 1m 38s
7:	learn: 0.5317194	total: 8.53s	remaining: 1m 38s
8:	learn: 0.5157837	total: 9.62s	remaining: 1m 37s
9:	learn: 0.5025932	total: 10.6s	remaining: 1m 34s
10:	learn: 0.4860133	total: 11.4s	remaining: 1m 32s
11:	learn: 0.4742485	total: 12.2s	remaining: 1m 29s
12:	learn: 0.4644863	total: 13.1s	remaining: 1m 27s
13:	learn: 0.4503247	total: 14s	remaining: 1m 26s
14:	learn: 0.4377393	total: 14.9s	remaining: 1m 24s
15:	learn: 0.4247406	total: 15.9s	remaining: 1m 23s
16:	learn: 0.4119508	total: 16.8s	remaining: 1m 22s
17:	learn: 0.4030521	total: 17.8s	remaining: 1m 21s
18:	learn: 0.3962178	total: 18.8s	remaining: 1m

In [39]:
model.score(test_data,test_labels)

0.5595667870036101

In [40]:
model.score(train_data,train_labels)

1.0

In [41]:
from sklearn.metrics import confusion_matrix

In [42]:
preds = model.predict(test_data)

In [43]:
conf = confusion_matrix(test_labels,preds)
conf

array([[ 53,  72],
       [ 50, 102]], dtype=int64)

In [44]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.5145631067961165

In [45]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5862068965517241

## Cross Validation

In [52]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date","Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [49]:
scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

Score : 0.5126353790613718 || TP : 0.5592105263157895 || TN : 0.456
Score : 0.5090252707581228 || TP : 0.5337837837837838 || TN : 0.4806201550387597
Score : 0.48014440433212996 || TP : 0.5272727272727272 || TN : 0.4107142857142857
Score : 0.4981949458483754 || TP : 0.5277777777777778 || TN : 0.46616541353383456
Score : 0.4693140794223827 || TP : 0.47282608695652173 || TN : 0.46236559139784944
Score : 0.5487364620938628 || TP : 0.5806451612903226 || TN : 0.5081967213114754
Score : 0.5415162454873647 || TP : 0.55 || TN : 0.5299145299145299
Score : 0.49097472924187724 || TP : 0.5563380281690141 || TN : 0.4222222222222222
Score : 0.4729241877256318 || TP : 0.5182926829268293 || TN : 0.40707964601769914
Score : 0.4981949458483754 || TP : 0.5189873417721519 || TN : 0.47058823529411764
Score : 0.5090252707581228 || TP : 0.5548387096774193 || TN : 0.45081967213114754
Score : 0.5523465703971119 || TP : 0.5903614457831325 || TN : 0.4954954954954955
Score : 0.48014440433212996 || TP : 0.519736842

In [50]:
from statistics import mean


print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [53]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.49458483754512633 || TP : 0.5474452554744526 || TN : 0.44285714285714284
Score : 0.48014440433212996 || TP : 0.506578947368421 || TN : 0.448
Score : 0.5126353790613718 || TP : 0.5535714285714286 || TN : 0.44954128440366975
Score : 0.51985559566787 || TP : 0.55 || TN : 0.48905109489051096
Score : 0.4548736462093863 || TP : 0.45977011494252873 || TN : 0.44660194174757284
Score : 0.48736462093862815 || TP : 0.5277777777777778 || TN : 0.44360902255639095
Score : 0.516245487364621 || TP : 0.5272727272727272 || TN : 0.5
Score : 0.516245487364621 || TP : 0.5827338129496403 || TN : 0.4492753623188406
Score : 0.49097472924187724 || TP : 0.5329341317365269 || TN : 0.42727272727272725
Score : 0.4548736462093863 || TP : 0.4797297297297297 || TN : 0.4263565891472868
Score : 0.49097472924187724 || TP : 0.5379746835443038 || TN : 0.42857142857142855
Score : 0.5631768953068592 || TP : 0.6 || TN : 0.5089285714285714
Score : 0.4620938628158845 || TP : 0.5033557046979866 || TN : 0.4140625
Score

In [54]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5050351510545317 || TP moyen : 0.539496997608193 || TN moyen : 0.4610381848716166


In [55]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.5451263537906137 || TP : 0.593103448275862 || TN : 0.49242424242424243
Score : 0.48736462093862815 || TP : 0.5144927536231884 || TN : 0.460431654676259
Score : 0.4981949458483754 || TP : 0.5522388059701493 || TN : 0.44755244755244755
Score : 0.49458483754512633 || TP : 0.5228758169934641 || TN : 0.4596774193548387
Score : 0.4981949458483754 || TP : 0.4935064935064935 || TN : 0.5040650406504065
Score : 0.5487364620938628 || TP : 0.5850340136054422 || TN : 0.5076923076923077
Score : 0.555956678700361 || TP : 0.5684931506849316 || TN : 0.5419847328244275
Score : 0.5234657039711191 || TP : 0.5850340136054422 || TN : 0.45384615384615384
Score : 0.5234657039711191 || TP : 0.5632911392405063 || TN : 0.47058823529411764
Score : 0.516245487364621 || TP : 0.5379310344827586 || TN : 0.49242424242424243
Score : 0.5415162454873647 || TP : 0.5833333333333334 || TN : 0.48760330578512395
Score : 0.5523465703971119 || TP : 0.6013513513513513 || TN : 0.49612403100775193
Score : 0.4945848375451

In [56]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5285958578757363 || TP moyen : 0.5643896808850625 || TN moyen : 0.48773372205756976
